In [1]:
import bisect

import pretty_midi as midi

In [2]:
# File handling
FILE_PATH = './files/midi/piano.mid'
OUT_FILE_PATH = './files/midi/piano-separated.mid'

# Defaults for every part and note
DEFAULT_INSTRUMENT = 'Acoustic Grand Piano'

# Read MIDI file and clean up
score = midi.PrettyMIDI(FILE_PATH)
score.remove_invalid_notes()
print('Loaded "{}".'.format(FILE_PATH))

Loaded "./files/midi/piano.mid".


In [3]:
class SortableNote(midi.Note):
    """Introduce a variant of the Note class to make it sortable."""
    
    def __init__(self, velocity, pitch, start, end):
        super().__init__(velocity, pitch, start, end)

    def __lt__(self, other):
        return self.start < other.start

In [4]:
# Get all notes and sort them by start time
notes = []
for instrument in score.instruments:
    for note in instrument.notes:
        # Convert Note to SortableNote
        notes.append(SortableNote(note.velocity,
                                  note.pitch,
                                  note.start,
                                  note.end))
notes.sort()

print('Found {} notes in whole score.'.format(len(notes)))

# Separating all notes in parts by checking if they overlap
parts = [notes]
part_index_offset = 0
movement_counter = 0

while part_index_offset < len(parts):
    part_notes = parts[part_index_offset]
    note_index = 0

    while len(part_notes) > 0 and note_index < len(part_notes):
        next_note_index = note_index + 1
        queue = []

        while (next_note_index < len(part_notes) - 1 and
               part_notes[next_note_index].start <= part_notes[note_index].end):
            queue.append(next_note_index)
            next_note_index += 1

        # Move notes which have been stored in a queue
        for index, move_note_index in enumerate(queue):
            part_index = part_index_offset + index + 1
            # Create part when it does not exist yet
            if len(parts) - 1 < part_index:
                parts.append([])
                
            # Move note to part
            note = part_notes[move_note_index]
            parts[part_index].append(note)
            movement_counter += 1
            
        # Remove notes from previous part
        if len(queue) == 1:
            del part_notes[queue[0]]
        elif len(queue) > 1:
            del part_notes[queue[0]:queue[-1]]

        # Start from top when we deleted something
        if len(queue) > 0:
            note_index = 0
        else:
            # .. otherwise move on to next note
            note_index += 1

    part_index_offset += 1

print('Created {} parts. Moved notes {} times.'.format(len(parts),
                                                       movement_counter))

Found 4767 notes in whole score.
Created 864 parts. Moved notes 25615 times.


In [5]:
# Merge parts when possible
print('Merging parts ..')
merged_counter = 0

for index, part in enumerate(reversed(parts)):
    part_index = len(parts) - index - 1
    queue = []
    
    for note_index, note in enumerate(part):
        done = False
        other_part_index = part_index - 1

        while not done:
            if other_part_index < 0:
                break
            
            other_note_index = -1
            found_free_space = True

            while True:
                other_note_index += 1
                
                # We reached the end .. nothing found!
                if other_note_index > len(parts[other_part_index]) - 1:
                    found_free_space = False
                    break

                other_note = parts[other_part_index][other_note_index]

                # Is there any overlapping notes?
                if not (note.end <= other_note.start or note.start >= other_note.end):
                    found_free_space = False
                    break

                # Stop here since there is nothing more coming.
                if other_note.start > note.end:
                     break

            if found_free_space:
                bisect.insort_left(parts[other_part_index], note)
                queue.append(note_index)
                merged_counter += 1
                done = True
            else:
                other_part_index -= 1
            
    # Delete moved notes from old part
    for index in sorted(queue, reverse=True):
        del part[index]
        
print('Done! Moved notes {} times for merging.'.format(merged_counter))

Merging parts ..
Done! Moved notes 195106 times for merging.


In [6]:
# Remove empty parts
remove_parts_queue = []
for part_index, part in enumerate(parts):
    if len(part) == 0:
        remove_parts_queue.append(part_index)
        
for index in sorted(remove_parts_queue, reverse=True):
    del parts[index]
    
print('Cleaned up {} empty parts after merging. Now {} parts.'.format(
    len(remove_parts_queue),
    len(parts)))

Cleaned up 816 empty parts after merging. Now 48 parts.


In [7]:
# Create a new MIDI file
new_score = midi.PrettyMIDI()

# Copy data from old score
new_score.time_signature_changes = score.time_signature_changes
new_score.key_signature_changes = score.key_signature_changes

# Create as many parts as we need to keep all voices separate
for instrument_index in range(0, len(parts)):
    program = midi.instrument_name_to_program(DEFAULT_INSTRUMENT)
    new_instrument = midi.Instrument(program=program)
    new_score.instruments.append(new_instrument)
    
# Assign notes to different parts
for part_index, part in enumerate(parts):
    for note in part:
        new_score.instruments[part_index].notes.append(note)
    print('Part #{} with {} notes.'.format(part_index + 1, len(part)))

Part #1 with 925 notes.
Part #2 with 920 notes.
Part #3 with 799 notes.
Part #4 with 786 notes.
Part #5 with 716 notes.
Part #6 with 605 notes.
Part #7 with 429 notes.
Part #8 with 305 notes.
Part #9 with 223 notes.
Part #10 with 151 notes.
Part #11 with 123 notes.
Part #12 with 88 notes.
Part #13 with 74 notes.
Part #14 with 57 notes.
Part #15 with 48 notes.
Part #16 with 43 notes.
Part #17 with 41 notes.
Part #18 with 39 notes.
Part #19 with 32 notes.
Part #20 with 32 notes.
Part #21 with 30 notes.
Part #22 with 25 notes.
Part #23 with 21 notes.
Part #24 with 18 notes.
Part #25 with 17 notes.
Part #26 with 14 notes.
Part #27 with 13 notes.
Part #28 with 11 notes.
Part #29 with 10 notes.
Part #30 with 9 notes.
Part #31 with 8 notes.
Part #32 with 8 notes.
Part #33 with 8 notes.
Part #34 with 8 notes.
Part #35 with 8 notes.
Part #36 with 8 notes.
Part #37 with 5 notes.
Part #38 with 5 notes.
Part #39 with 4 notes.
Part #40 with 4 notes.
Part #41 with 4 notes.
Part #42 with 4 notes.
Par

In [8]:
# Save result
new_score.write(OUT_FILE_PATH)